In [37]:
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D,LeakyReLU,BatchNormalization,GlobalAveragePooling2D,GlobalMaxPooling2D,AveragePooling2D
from keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, initializers
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications import DenseNet121

In [38]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,              
    restore_best_weights=True
)

In [39]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/aiderdata/AIDER",
    validation_split=0.15,
    subset="training",
    seed=123,
    image_size=(300, 300),
    batch_size=64
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/aiderdata/AIDER",
    validation_split=0.15,
    subset="validation",
    seed=123,
    image_size=(300, 300),
    batch_size=64
)

Found 6433 files belonging to 5 classes.
Using 5469 files for training.
Found 6433 files belonging to 5 classes.
Using 964 files for validation.


In [40]:
from tensorflow.keras import layers
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),         
    layers.RandomRotation(0.25),             
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomContrast(0.1)
])

In [41]:
normalization_layer = layers.Rescaling(1./255)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x, y: (data_augmentation(normalization_layer(x)), y), num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [42]:
weights_Add="/kaggle/input/efficientnet/efficientnetb3_notop.h5"
base_model = EfficientNetB3(weights=weights_Add, include_top=False, input_shape=(300, 300, 3))
base_model.trainable = False

In [43]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(64),
    LeakyReLU(alpha=0.1),
    Dense(5, activation='softmax')  
])

In [44]:
model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)     │ (None, 10, 10, 1536)   │    10,783,535 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 153600)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │     9,830,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,614,324 (78.64 MB)

 Trainable params: 9,830,789 (37.50 MB)

 Non-trainable params: 10,783,535 (41.14 MB)

In [45]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_ds,epochs=30,validation_data=val_ds,callbacks=[early_stop])

Epoch 1/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.4731 - loss: 29.2176 - val_accuracy: 0.0799 - val_loss: 9.8715
Epoch 2/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 57s 642ms/step - accuracy: 0.4905 - loss: 5.3074 - val_accuracy: 0.0747 - val_loss: 6.8192
Epoch 3/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 57s 639ms/step - accuracy: 0.4715 - loss: 5.9406 - val_accuracy: 0.6888 - val_loss: 4.3151
Epoch 4/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 57s 640ms/step - accuracy: 0.5218 - loss: 5.5310 - val_accuracy: 0.0820 - val_loss: 1.8129
Epoch 5/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 57s 642ms/step - accuracy: 0.4781 - loss: 5.6250 - val_accuracy: 0.6867 - val_loss: 2.7004
Epoch 6/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 57s 642ms/step - accuracy: 0.5546 - loss: 1.7691 - val_accuracy: 0.6867 - val_loss: 2.8459
Epoch 7/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 57s 637ms/step - accuracy: 0.5508 - loss: 2.0395 - val_accuracy: 0.6846 - val_loss: 2.0678
Epoch 8/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 57s 635ms/step - accuracy: 0.4837 - loss: 3.0648 - val_accur